In [1]:
import numpy as np
import torch
from torch import nn
import torchvision as tv
import time
from torchvision import datasets, models, transforms
import os

C:\ProgramData\Anaconda3\envs\myenv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [3]:
BATCH_SIZE = 4

data_dir= '../input/hymenoptera_data/hymenoptera_data'

train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'), data_transforms['train'])
test_dataset = datasets.ImageFolder(os.path.join(data_dir, 'val'), data_transforms['val'])
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [40]:
evaluate_accuracy(test_iter, model)

tensor([[ 1.1534, -0.6740],
        [ 0.6922, -0.5466],
        [ 1.3871, -0.0320],
        [ 1.5880, -1.2704]], grad_fn=<AddmmBackward0>)
tensor([0, 0, 0, 0])
tensor([[ 6.2296, -5.5126],
        [ 4.1795, -1.3518],
        [-5.2100,  4.7924],
        [ 1.7758,  0.0703]], grad_fn=<AddmmBackward0>)
tensor([0, 0, 0, 0])
tensor([[ 2.7469, -1.9797],
        [ 3.8616, -3.3273],
        [ 5.1585, -3.9227],
        [ 5.5832, -3.2029]], grad_fn=<AddmmBackward0>)
tensor([0, 0, 0, 0])
tensor([[ 5.5474, -3.4183],
        [ 0.8089,  1.4567],
        [ 4.5543, -2.8858],
        [ 7.4123, -6.4605]], grad_fn=<AddmmBackward0>)
tensor([0, 0, 0, 0])
tensor([[ 5.9560, -4.8910],
        [ 0.1199,  1.2136],
        [ 1.8279, -0.4323],
        [ 3.4962, -2.1025]], grad_fn=<AddmmBackward0>)
tensor([0, 0, 0, 0])
tensor([[ 1.7365, -0.5570],
        [ 3.6902, -3.9496],
        [-1.4588,  2.9625],
        [-0.2186,  0.2968]], grad_fn=<AddmmBackward0>)
tensor([0, 0, 0, 0])
tensor([[ 3.9215, -1.8141],
        [ 3.

0.9150326797385621

In [39]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = torch.Tensor([0]), 0
    net.eval()
    for X, y in data_iter:
        print(net(X))
        print(y)
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n

In [21]:
def train(net, train_iter, test_iter, trainer, num_epochs):
    loss = nn.CrossEntropyLoss(reduction='sum')
    net.train()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            trainer.zero_grad()
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]
            # print("Step. time since epoch: {:.3f}. Train acc: {:.3f}. Train Loss: {:.3f}".format(time.time() -  start,
            #     (y_hat.argmax(axis=1) == y).sum().item() / y.shape[0], l.item()))
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, '
              'time %.1f sec'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc,
                 time.time() - start))

In [6]:
def vgg_block(num_convs, input_channels, num_channels):

    block = nn.Sequential(
        nn.Conv2d(input_channels, num_channels, kernel_size=3, padding=1),
        nn.ReLU()
    )

    for i in range(num_convs - 1):
        block.add_module("conv{}".format(i),
                         nn.Conv2d(num_channels, num_channels, kernel_size=3, padding=1)
                         )
        block.add_module("relu{}".format(i),
                         nn.ReLU()
                         )

    block.add_module("pool", nn.MaxPool2d(2, stride=2))

    return block

In [7]:
conv_arch = ((1, 1, 64), (1, 64, 128), (2, 128, 256), (2, 256, 512), (2, 512, 512))

In [8]:
def vgg(conv_arch):
    net = nn.Sequential()

    for i, (num_convs, input_ch, num_channels) in enumerate(conv_arch):
        net.add_module("block{}".format(i), vgg_block(num_convs, input_ch, num_channels))


    classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(25088, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, 2))

    net.add_module('classifier', classifier)
    return net

net = vgg(conv_arch)

In [33]:
net = models.vgg16()
net

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [43]:


model_vgg16 = models.vgg16()



In [44]:
model_vgg16

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [41]:
model = models.resnet18(pretrained=True)

C:\ProgramData\Anaconda3\envs\myenv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\myenv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [42]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [15]:

for param in model.parameters():
    param.requires_grad = False

In [19]:
model.fc = nn.Linear(in_features=512, out_features=2)

In [37]:
# model_vgg16.classifier = torch.nn.Linear(25088, 2)

In [40]:
# layers_to_freeze = 30
# for i, (name, param) in enumerate(model_vgg16.features.named_parameters()):
#     if i < layers_to_freeze:
#         param.requires_grad = False
#     print(f'{name:30}{param.requires_grad}')

0.weight                      False
0.bias                        False
2.weight                      False
2.bias                        False
5.weight                      False
5.bias                        False
7.weight                      False
7.bias                        False
10.weight                     False
10.bias                       False
12.weight                     False
12.bias                       False
14.weight                     False
14.bias                       False
17.weight                     False
17.bias                       False
19.weight                     False
19.bias                       False
21.weight                     False
21.bias                       False
24.weight                     False
24.bias                       False
26.weight                     False
26.bias                       False
28.weight                     False
28.bias                       False


In [22]:
lr, num_epochs = 0.001, 50
trainer = torch.optim.Adam(model.parameters(), lr=lr)
train(model, train_iter, test_iter, trainer, num_epochs)

epoch 1, loss 1.9659, train acc 0.631, test acc 0.582, time 13.4 sec
epoch 2, loss 0.6339, train acc 0.775, test acc 0.758, time 12.7 sec
epoch 3, loss 0.3431, train acc 0.877, test acc 0.719, time 13.1 sec
epoch 4, loss 0.3644, train acc 0.889, test acc 0.778, time 12.6 sec
epoch 5, loss 0.3332, train acc 0.873, test acc 0.745, time 12.5 sec
epoch 6, loss 0.3126, train acc 0.861, test acc 0.712, time 12.6 sec
epoch 7, loss 0.2946, train acc 0.885, test acc 0.863, time 12.5 sec
epoch 8, loss 0.2970, train acc 0.889, test acc 0.758, time 12.5 sec
epoch 9, loss 0.2695, train acc 0.885, test acc 0.843, time 12.6 sec
epoch 10, loss 0.2550, train acc 0.902, test acc 0.824, time 12.6 sec
epoch 11, loss 0.2572, train acc 0.865, test acc 0.830, time 12.6 sec
epoch 12, loss 0.2617, train acc 0.885, test acc 0.850, time 12.7 sec
epoch 13, loss 0.2018, train acc 0.922, test acc 0.902, time 12.6 sec
epoch 14, loss 0.2056, train acc 0.934, test acc 0.869, time 12.6 sec
epoch 15, loss 0.1595, train 

In [31]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [26]:
model_vgg = tv.models.vgg16(pretrained=True)

C:\ProgramData\Anaconda3\envs\myenv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\myenv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [27]:
model_vgg

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [30]:
import numpy as np
np.array([[1],[3], [6]]).argmax(axis=0)

array([2], dtype=int64)

In [36]:
np.array([[ 1.1534, -0.6740],
        [ 0.6922, -0.5466],
        [ 1.3871, 2.0320],
        [ 1.5880, -1.2704]]).argmax(axis=1)

array([0, 0, 1, 0], dtype=int64)